# Importing Libraries 

#### To ensure compatibility with the MIA library, we are utilizing Tensorflow 2.5 and numpy 1.19 since the MIA library is not compatible with higher versions of Tensorflow.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import random

import copy
import os
from sklearn.model_selection import train_test_split
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import torchvision.transforms as transform
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data

 
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers

from tensorflow_privacy.privacy.optimizers import dp_optimizer_keras
import tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
print(np.__version__)

### Reading data

In [ ]:
# Load training data from 'X_train.npy'
X_train = np.load('X_train.npy')
# Load testing data from 'X_test.npy'
X_test = np.load('X_test.npy')

# Load training labels from 'y_train.npy'
y_train = np.load('y_train.npy')
# Load testing labels from 'y_test.npy'
y_test = np.load('y_test.npy')



print("Shape of X_train:", X_train.shape) # Print the shape of X_train array
print("Shape of X_test:", X_test.shape) # Print the shape of X_test array
print("Shape of y_train:", y_train.shape) # Print the shape of y_train array
print("Shape of y_test:", y_test.shape) # Print the shape of y_test array

In [ ]:
#Since the membership inference attack accepts data in multiples of the batch size, 
#we will access the first 5000 elements for X_train and y_train

X_train = X_train[0:5000]
y_train = y_train[0:5000]


print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### Using keras data loader to load numpy arrays

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()
train_generator = datagen.flow(X_train, y_train, batch_size=10)
test_generator = datagen.flow(X_test, y_test, batch_size=10)

### Non Differentially Private Models

In [ ]:
def target_model_fn_keras():
    """
    Creates and compiles a target model based on the DenseNet121 architecture.

    Returns:
    - model: Compiled target model
    """

    # Create a new sequential model
    model = Sequential() 

    # Load the pre-trained DenseNet121 model with weights from 'DenseNet-BC-121-32-no-top.h5'
    base_model = DenseNet121(weights='DenseNet-BC-121-32-no-top.h5', include_top=False) 

    # Add the DenseNet121 model to the new sequential model
    model.add(base_model)

    # Add a global average pooling layer
    model.add(GlobalAveragePooling2D())

    # Add a dense output layer with the appropriate number of classes (2 classes in this case)
    model.add(Dense(2, activation='softmax'))

    # Define the loss function as CategoricalCrossentropy with logits and no reduction
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)

    # Compile the model with Adam optimizer, the defined loss function, and accuracy as the evaluation metric
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    # Return the compiled model
    return model


def attack_model_fn_keras():
    """
    Creates and compiles an attack model based on a dense architecture.

    Returns:
    - model: Compiled attack model
    """

    # Create a new sequential model
    model = tf.keras.models.Sequential()

    # Add a dense layer with 128 units and ReLU activation function
    model.add(layers.Dense(128, activation="relu", input_shape=(2,)))

    # Apply dropout regularization with a rate of 0.3
    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))

    # Add a dense layer with 64 units and ReLU activation function
    model.add(layers.Dense(64, activation="relu"))

    # Apply dropout regularization with a rate of 0.2
    model.add(layers.Dropout(0.2, noise_shape=None, seed=None))

    # Add another dense layer with 64 units and ReLU activation function
    model.add(layers.Dense(64, activation="relu"))

    # Add a dense output layer with 1 unit and sigmoid activation function
    model.add(layers.Dense(1, activation="sigmoid"))

    # Compile the model with Adam optimizer, binary cross-entropy loss function, and accuracy metric
    model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])

    # Return the compiled model
    return model


In [ ]:
## Create an EarlyStopping callback to stop training if the loss does not improve for 3 consecutive epochs
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

## Print a message indicating that the target model training is starting
print("Training the target model with keras model")

## Create the target model using the target_model_fn_keras() function
target_model = target_model_fn_keras()

## Fit the target model using the training generator
## Train for 18 epochs, display verbose output, and use the EarlyStopping callback
## Store the training history in the history_target_model variable
history_target_model = target_model.fit_generator(
    train_generator, epochs=18, verbose=True, callbacks=[callback]
)


In [ ]:
## Predict the labels for the test set using the target model
y_test_hat = target_model.predict(X_test, batch_size=4)

## Convert the predicted probabilities to class labels by selecting the index with the highest probability
y_test_hat = np.argmax(y_test_hat, axis=1)

## Convert the original labels from one-hot encoding to class labels
y_test_org = np.argmax(y_test, axis=1)

## Calculate the confusion matrix and classification report
conf_m = confusion_matrix(y_test_org, y_test_hat)
clas_r = classification_report(y_test_org, y_test_hat)

## Plot the confusion matrix as a heatmap
plt.figure(figsize=(5,4))
sns.set(font_scale=1.2)
ax = sns.heatmap(conf_m, annot=True, xticklabels=['H', 'P'], yticklabels=['H', 'P'], cbar=False, cmap='Blues', linewidths=1, linecolor='black', fmt='.0f')
plt.yticks(rotation=0)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
ax.xaxis.set_ticks_position('top') 
plt.title('Confusion matrix - test data\n(H - healthy/normal, P - pneumonia)')
plt.show()

## Print the classification report on the test data for the Non DP Model
print('Classification report on test data for Non DP Model')
print(clas_r)

## Calculate the accuracy score
acc_score = accuracy_score(y_test_org, y_test_hat)
print(acc_score)


In [ ]:
## Create a ShadowModelBundle with the specified target model function, shadow dataset size, and number of models
smb = ShadowModelBundle(
    target_model_fn_keras,
    shadow_dataset_size=200,
    num_models=5,
)

## Split the attacker dataset into training and test sets
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(X_test, y_test, test_size=0.1)

## Print the shapes of the attacker training and test sets
print(attacker_X_train.shape, attacker_X_test.shape)

## Take the first 500 samples from the attacker training set
attacker_X_train = attacker_X_train[0:500]
attacker_y_train = attacker_y_train[0:500]

## Take the first 60 samples from the attacker test set
attacker_X_test = attacker_X_test[0:60]
attacker_y_test = attacker_y_test[0:60]

## Fit the ShadowModelBundle on the attacker training data and transform it
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=10,
        verbose=False,
        validation_data=(attacker_X_test, attacker_y_test),
        batch_size=10
    ),
)

## Create an AttackModelBundle with the specified attack model function and number of classes
amb = AttackModelBundle(attack_model_fn_keras, num_classes=2)

## Fit the attack models using the shadow data
amb.fit(
    X_shadow, y_shadow, fit_kwargs=dict(epochs=10, verbose=False)
)

## Define the size of the attack test dataset
ATTACK_TEST_DATASET_SIZE = 200

## Prepare the attack data in the expected format for the AttackModelBundle
data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
data_out = X_test[:ATTACK_TEST_DATASET_SIZE], y_test[:ATTACK_TEST_DATASET_SIZE]

## Compile the attack test data and real membership labels
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

## Predict the membership labels using the AttackModelBundle
attack_guesses = amb.predict(attack_test_data)

## Compute the attack accuracy by comparing the predicted labels with the real membership labels
attack_accuracy = np.mean(attack_guesses == real_membership_labels)


In [ ]:
attack_accuracy

### Differentially Private Models

In [ ]:
def mia_attack(noise_multiplier,l2_norm_clip ):
    
    """
    Perform MIA attack.

    Args:
    - noise_multiplier: Noise multiplier for DP optimizer
    - l2_norm_clip: L2 norm clip for DP optimizer

    Returns:
    - attack_accuracy: Attack accuracy
    - epsilon: Privacy parameter epsilon
    - acc_score: Accuracy score
    
    """
    
    def target_model_fn_keras_dp():
        
        """
        Creates and compiles a target model with Differential Privacy (DP) optimizer.

        Returns:
        - model: Compiled target model
        
        """
        
        ## Load the pre-trained DenseNet121 model with weights
        base_model = DenseNet121(weights='DenseNet-BC-121-32-no-top.h5', include_top=False, input_shape = (224, 224, 3))

        # Create a new sequential model
        model = Sequential()

        # Add the DenseNet121 model to the new sequential model
        model.add(base_model)

        # Add a global average pooling layer
        model.add(GlobalAveragePooling2D())

        # Add a dense output layer with the appropriate number of classes
        model.add(Dense(2, activation='softmax'))

        # Define the loss function as CategoricalCrossentropy with logits and no reduction
        loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
    

        # Create a DP optimizer with specified parameters
        optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=10,
        learning_rate=0.25)

        
        # Compile the model with the DP optimizer, loss function, and accuracy metric
        model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

        return model
    
     

    def attack_model_fn_keras_dp():
        
        """
        Creates and compiles an attack model with Differential Privacy (DP) optimizer.

        Returns:
        - model: Compiled attack model
        """
        
        # Create a new sequential model
        model = tf.keras.models.Sequential()
        
        # Add a dense layer with 128 units and ReLU activation function
        model.add(layers.Dense(128, activation="relu", input_shape=(2,)))

        # Apply dropout regularization with a rate of 0.3
        model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
        
        # Add a dense layer with 64 units and ReLU activation function
        model.add(layers.Dense(64, activation="relu"))
        
        # Apply dropout regularization with a rate of 0.2
        model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
        
        # Add another dense layer with 64 units and ReLU activation function
        model.add(layers.Dense(64, activation="relu"))
        
        # Add a dense output layer with 1 unit and sigmoid activation function
        model.add(layers.Dense(1, activation="sigmoid"))
        
        # Compile the model with Adam optimizer, binary cross-entropy loss function, and accuracy metric
        model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
        
        return model  
    
    
    ## Create the target model using the target_model_fn_keras() function
    target_model = target_model_fn_keras_dp()
    
    ## Create an EarlyStopping callback to stop training if the loss does not improve for 3 consecutive epochs
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    

    ## Fit the target model using the training generator
    ## Train for 18 epochs, display verbose output, and use the EarlyStopping callback
    ## Store the training history in the history_target_model variable
    history_target_model = target_model.fit_generator(
            train_generator, epochs=18, verbose=True, callbacks=[callback]
        )
    

    
    ## Predict the labels for the test set using the target model
    y_test_hat = target_model.predict(X_test, batch_size=4)
    
    ## Convert the predicted probabilities to class labels by selecting the index with the highest probability
    y_test_hat = np.argmax(y_test_hat, axis=1)
    
    ## Convert the original labels from one-hot encoding to class labels
    y_test_org = np.argmax(y_test, axis=1)
    
    ## Calculate the accuracy score
    acc_score = accuracy_score(y_test_org, y_test_hat)
    

    ## Create a ShadowModelBundle with the specified target model function, shadow dataset size, and number of models
    smb = ShadowModelBundle(
        target_model_fn_keras_dp,
        shadow_dataset_size=200,
        num_models=5,
    )
    
    ## Split the attacker dataset into training and test sets
    attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(X_test, y_test, test_size=0.1
    )
    
    ## Print the shapes of the attacker training and test sets
    print(attacker_X_train.shape, attacker_X_test.shape)
    
    ## Take the first 500 samples from the attacker training set
    attacker_X_train = attacker_X_train[0:500]
    attacker_y_train = attacker_y_train[0:500]

    ## Fit the ShadowModelBundle on the attacker training data and transform it
    attacker_X_test =   attacker_X_test[0:60]
    attacker_y_test =   attacker_y_test[0:60]
    
    
    ## print("Training the shadow models...")

    
    X_shadow, y_shadow = smb.fit_transform(
        attacker_X_train,
        attacker_y_train,
        fit_kwargs=dict(
            epochs=10,
            verbose=False,
            validation_data=(attacker_X_test, attacker_y_test),
             batch_size = 10
        ),
    )
    
    ## Create an AttackModelBundle with the specified attack model function and number of classes
    amb = AttackModelBundle(attack_model_fn_keras_dp, num_classes=2)

    ## Fit the attack models using the shadow data
    amb.fit(
            X_shadow, y_shadow, fit_kwargs=dict(epochs=10, verbose=False)
        )
    
    ## Define the size of the attack test dataset
    ATTACK_TEST_DATASET_SIZE = 200
    
    ## Prepare the attack data in the expected format for the AttackModelBundle
    data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
    data_out = X_test[:ATTACK_TEST_DATASET_SIZE], y_test[:ATTACK_TEST_DATASET_SIZE]

    ## Compile the attack test data and real membership labels
    attack_test_data, real_membership_labels = prepare_attack_data(
            target_model, data_in, data_out
        )

    ## Predict the membership labels using the AttackModelBundle
    attack_guesses = amb.predict(attack_test_data)
    
    ## Compute the attack accuracy by comparing the predicted labels with the real membership labels
    attack_accuracy = np.mean(attack_guesses == real_membership_labels)
    
    
    ## Compute privacy parameter epsilon using the compute_dp_sgd_privacy function
    epsilon =  compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=X_train.shape[0],
                                              batch_size=5,
                                              noise_multiplier=noise_multiplier,
                                              epochs=15,
                                              delta=0.0001)

    return attack_accuracy, epsilon, acc_score

    


### Noise Multiplier = 0.001

In [ ]:
mia_acc = []
epsilon = []
accuracy = []

# Run the MIA attack 5 times with the selected noise multiplier
for i in range(5):
    ans =  mia_attack(0.001,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

# Extract the epsilon values from the results
epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.0025

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.0025,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.005

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.005,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.006

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.006,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.008

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.008,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.01

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(3):
    ans =  mia_attack(0.01,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.02

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.02,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.03

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.03,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.04


In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.04,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.05

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.05,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.1

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.1,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 0.5

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(0.5,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

### Noise Multiplier = 1

In [ ]:
mia_acc = []
epsilon = []
accuracy = []
for i in range(5):
    ans =  mia_attack(1,1.5)
    mia_acc.append(ans[0])
    print('MIA Accuracy')
    print(ans[0])
    print()
    print('Epsilon')
    print(ans[1])
    print()
    print('Accuracy')
    print(ans[2])
    epsilon.append(ans[1])
    accuracy.append(ans[2])

epsilon_list = list(zip(*epsilon))[0]

## Analysis with Densenet

### MIA Accuracy

In [ ]:
##0.0025
n1 = np.array([0.6175,0.5675, 0.58,0.5625, 0.595])

##0.005
n2 = np.array([0.5875, 0.565, 0.5425, 0.595, 0.5])

##0.01
n3 = np.array([0.6, 0.565, 0.585, 0.55, 0.6])

##0.05
n4 = np.array([0.3675, 0.5, 0.4625, 0.5, 0.5])


##0.1
n5 = np.array([0.555, 0.56, 0.5, 0.555, 0.445])


# Calculate the average
n1_mean = np.mean(n1)
n2_mean = np.mean(n2)
n3_mean = np.mean(n3)
n4_mean = np.mean(n4)
n5_mean = np.mean(n5)


# Calculate the standard deviation
n1_std = np.std(n1)
n2_std =np.std(n2)
n3_std =  np.std(n3)
n4_std = np.std(n4)
n5_std =  np.std(n5)


In [ ]:
#labels = [0.01, 0.05, 0.1, 0.5, 1]

labels = [0.0025, 0.005, 0.01, 0.05, 0.1]
labels_log = np.log([0.0025, 0.005, 0.01, 0.05, 0.1])
x_pos = np.arange(len(labels))
CTEs = [n1_mean, n2_mean, n3_mean, n4_mean, n5_mean]
error = [n1_std, n2_std, n3_std, n4_std, n5_std ]

In [ ]:
fig, ax = plt.subplots()
ax.bar(labels_log, CTEs,
       yerr=error,
       align='center',
       alpha=0,
       ecolor='black',
       capsize=10)

ax.set_xticks(labels_log)
ax.set_xticklabels(labels)
ax.set_title('Noise Multiplier vs MIA Accuracy')
ax.set_xlabel('Noise Multiplier')
ax.set_ylabel('Average MIA Accuracy')
ax.yaxis.grid(True)
ax.plot( labels_log, CTEs)
ax.plot( labels_log, CTEs, 'ro')
# Save the figure and show
plt.tight_layout()
# plt.savefig('bar_plot_with_error_bars.png')
plt.show()

### Testing Accuracy

In [ ]:
##0.0025
n1 = np.array( [0.8668,0.8635,0.8539, 0.900481, 0.9325])

##0.005
n2 = np.array( [0.84109, 0.865168539, 0.91011236, 0.91011236, 0.37720706260032105])

##0.01
n3 = np.array( [0.825, 0.841, 0.801, 0.9117, 0.7223])

##0.05
n4 = np.array([0.7479935794542536, 0.43820224719101125, 0.8218298555377207, 0.6260032102728732, 0.37720706260032105])


##0.1
n5 = np.array([0.6260032102728732,0.6179775280898876,0.6260032102728732,0.6260032102728732,0.6260032102728732])


# Calculate the average
n1_mean = np.mean(n1)
n2_mean = np.mean(n2)
n3_mean = np.mean(n3)
n4_mean = np.mean(n4)
n5_mean = np.mean(n5)


# Calculate the standard deviation
n1_std = np.std(n1)
n2_std =np.std(n2)
n3_std =  np.std(n3)
n4_std = np.std(n4)
n5_std =  np.std(n5)


In [ ]:
#labels = [0.01, 0.05, 0.1, 0.5, 1]

labels = [0.0025, 0.005, 0.01, 0.05, 0.1]
labels_log = np.log([0.0025, 0.005, 0.01, 0.05, 0.1])
x_pos = np.arange(len(labels))
CTEs = [n1_mean, n2_mean, n3_mean, n4_mean, n5_mean]
error = [n1_std, n2_std, n3_std, n4_std, n5_std ]

In [ ]:
fig, ax = plt.subplots()
ax.bar(labels_log, CTEs,
       yerr=error,
       align='center',
       alpha=0,
       ecolor='black',
       capsize=10)

ax.set_xticks(labels_log)
ax.set_xticklabels(labels)
ax.set_title('Noise Multiplier vs MIA Accuracy')
ax.set_xlabel('Noise Multiplier')
ax.set_ylabel('Average MIA Accuracy')
ax.yaxis.grid(True)
ax.plot( labels_log, CTEs)
ax.plot( labels_log, CTEs, 'ro')
# Save the figure and show
plt.tight_layout()
# plt.savefig('bar_plot_with_error_bars.png')
plt.show()

In [ ]:
noise_multiplier = [0.0010, 0.0025,0.0050, 0.0060,0.0080, 0.0100, 0.0200, 0.0300, 0.0400,0.0500]  
eps = []
for i in range(len(noise_multiplier)):
    
    epsilon =  compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=X_train.shape[0],
                                                  batch_size=10,
                                                  noise_multiplier=noise_multiplier[i],
                                                  epochs=25,
                                                  delta=0.0001)
    eps.append(epsilon[0])

In [ ]:
eps